In [1]:
from autograd import grad
import autograd.numpy as np
import matplotlib.pyplot as plt
from lista import LISTA, make_loss
from functions import *
from scipy.linalg import hadamard

In [6]:
n_samples = 10000
n_test = 1000
n = 5
p = 10
rng = np.random.RandomState(0)

fit_loss = 'l2'
reg = 'l1'

fit_function, der_function = {
                              'l2': (l2_fit, l2_der),
                              'logreg': (logreg_fit, logreg_der)
                              }[fit_loss]
reg_function, prox = {
                      'l2': (l2_pen, l2_prox),
                      'l1': (l1_pen, l1_prox),
                      None: (no_pen, no_prox)
                      }[reg]
loss = make_loss(fit_function, reg_function)



K = np.eye(p)

z_true = rng.randn(p, n_samples)
D = rng.randn(n, p).dot(K)
D /= np.linalg.norm(D, axis=0, keepdims=True)

sigma = 0.1
eps = rng.randn(n, n_samples)

z_test = rng.randn(p, n_test)
eps_test = rng.randn(n, n_test)
X = np.dot(D, z_true) + eps
X_test = np.dot(D, z_test) + eps_test
if fit_loss == 'logreg':
    X = 2 * (X > 0) - 1
    X_test = 2 * (X_test > 0) - 1

In [7]:
lbda_max = np.max(np.abs(D.T.dot(X)))
lbda = .6 * lbda_max
print(lbda_max)

10.261304163592998


In [8]:
f_min = LISTA(D, lbda, 1000, fit_loss, reg).compute_loss(X_test)

In [9]:
n_layers = 3
lista = LISTA(D, lbda, n_layers, fit_loss, reg, variables='both', learn_levels=False)
z = lista.transform(X_test)
print('frac: %.2e' % (np.sum(np.sum(z != 0 , axis=0) > 0) / X_test.size))
print('loss: %.2e' % (lista.compute_loss(X_test) - f_min))
lista.fit(X, l_rate=1e-1, max_iter=200, verbose=100, backtrack=True)
z = lista.transform(X_test)
print('frac: %.2e' % (np.sum(np.sum(z != 0 , axis=0) > 0) / X_test.size))
print('loss: %.2e' % (lista.compute_loss(X_test) - f_min))

frac: 2.00e-03
loss: 4.79e-04
it 0, loss = 7.449e+00, grad_W = 3.46e-02, grad_l = 2.82e-02, , l_rate = 1.00e-01
it 100, loss = 7.448e+00, grad_W = 2.35e-03, grad_l = 5.05e-03, , l_rate = 8.00e-01
frac: 3.40e-03
loss: 4.24e-04


In [11]:
U, S, V = np.linalg.svd(D, full_matrices=False)

In [37]:
2.09 * (S[0] / S[1]) ** 2

4.8093331696126365

In [32]:
direction = V[:, 1]

In [33]:
x_train = np.random.randn(n_samples)[:, None] * direction[None, :]

In [34]:
lista = LISTA(D, 0.1, n_layers, fit_loss, reg, variables='W1', learn_levels=False)

In [35]:
lista.fit(x_train.T, verbose=10, max_iter=100)

it 0, loss = 1.007e-01, grad_W = 2.09e-01, grad_l = 2.99e-02, , l_rate = 1.00e-03
it 10, loss = 1.005e-01, grad_W = 2.08e-01, grad_l = 2.98e-02, , l_rate = 1.00e-03
it 20, loss = 1.004e-01, grad_W = 2.07e-01, grad_l = 2.96e-02, , l_rate = 1.00e-03
it 30, loss = 1.003e-01, grad_W = 2.06e-01, grad_l = 2.95e-02, , l_rate = 1.00e-03
it 40, loss = 1.002e-01, grad_W = 2.05e-01, grad_l = 2.94e-02, , l_rate = 1.00e-03
it 50, loss = 1.001e-01, grad_W = 2.04e-01, grad_l = 2.92e-02, , l_rate = 1.00e-03
it 60, loss = 9.995e-02, grad_W = 2.03e-01, grad_l = 2.91e-02, , l_rate = 1.00e-03
it 70, loss = 9.983e-02, grad_W = 2.02e-01, grad_l = 2.90e-02, , l_rate = 1.00e-03
it 80, loss = 9.972e-02, grad_W = 2.01e-01, grad_l = 2.88e-02, , l_rate = 1.00e-03
it 90, loss = 9.961e-02, grad_W = 2.00e-01, grad_l = 2.87e-02, , l_rate = 1.00e-03
